In [1]:
import pandas as pd
customer_churn = pd.read_csv("customer_churn.csv")
data = pd.DataFrame(customer_churn)


In [2]:
#2.列出資料筆數、屬性數量、每個欄位的空值個數以及各類別(target)的資料筆數

data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3083 entries, 0 to 3082
Data columns (total 20 columns):
CustomerID                     3083 non-null int64
Churn                          3083 non-null int64
Tenure                         2930 non-null float64
PreferredLoginDevice           3083 non-null object
CityTier                       3083 non-null int64
WarehouseToHome                2929 non-null float64
PreferredPaymentMode           3083 non-null object
Gender                         3083 non-null object
HourSpendOnApp                 2933 non-null float64
NumberOfDeviceRegistered       3083 non-null int64
PreferedOrderCat               3083 non-null object
SatisfactionScore              3083 non-null int64
MaritalStatus                  3083 non-null object
NumberOfAddress                3083 non-null int64
Complain                       3083 non-null int64
OrderAmountHikeFromlastYear    2952 non-null float64
CouponUsed                     2957 non-null float64
OrderCount 

In [3]:
sum =  data.isna().sum()
print("--------各欄位空值---------")
print(sum)

--------各欄位空值---------
CustomerID                       0
Churn                            0
Tenure                         153
PreferredLoginDevice             0
CityTier                         0
WarehouseToHome                154
PreferredPaymentMode             0
Gender                           0
HourSpendOnApp                 150
NumberOfDeviceRegistered         0
PreferedOrderCat                 0
SatisfactionScore                0
MaritalStatus                    0
NumberOfAddress                  0
Complain                         0
OrderAmountHikeFromlastYear    131
CouponUsed                     126
OrderCount                     128
DaySinceLastOrder              166
CashbackAmount                   0
dtype: int64


In [4]:
"""
3.資料集中有部分資料重覆出現，請刪除重覆多餘的資料 (僅保留一筆)，並列出剩
餘的資料筆數
"""
data.drop_duplicates(inplace=True)
len(data.index)

3078

In [5]:
#4. 填補空值及其他資料前處理 (e.g., feature encoding) 
data["Tenure"]=data['Tenure'].fillna(data['Tenure'].mean())
data['WarehouseToHome']=data['WarehouseToHome'].fillna(data['WarehouseToHome'].mean())
data['HourSpendOnApp']=data['HourSpendOnApp'].fillna(data['HourSpendOnApp'].mode()[0])
data['OrderAmountHikeFromlastYear']=data['OrderAmountHikeFromlastYear'].fillna(data['OrderAmountHikeFromlastYear'].mean())
data['CouponUsed']=data['CouponUsed'].fillna(method='pad')
data['OrderCount']=data['OrderCount'].fillna(method='pad')
data['DaySinceLastOrder']=data['DaySinceLastOrder'].fillna(method='pad')

In [6]:
#4. 填補空值及其他資料前處理 (e.g., feature encoding) 

from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
data['PreferredLoginDevice'] = labelencoder.fit_transform(data['PreferredLoginDevice'])
data['PreferredPaymentMode'] = labelencoder.fit_transform(data['PreferredPaymentMode'])
data['Gender'] = labelencoder.fit_transform(data['Gender'])
data['PreferedOrderCat'] = labelencoder.fit_transform(data['PreferedOrderCat'])
data['MaritalStatus'] = labelencoder.fit_transform(data['MaritalStatus'])



In [7]:
#5. 將資料集劃分為訓練集與測試集

from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import accuracy_score 

X=data.drop(['Churn'], axis=1)
y=data['Churn']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size= 0.25,random_state=12)


In [10]:
#6. 訓練與測試 SVM、Logistic Regression、Decision Tree 模型
from sklearn import svm
clf = svm.SVC(kernel='linear',C = 10) 
clf.fit(X_train, y_train.astype('int')) 
y_pred = clf.predict(X_test)
svm_accuracy=accuracy_score(y_test,y_pred)

from sklearn.linear_model import LogisticRegression
clf1=LogisticRegression(fit_intercept=True,C=1.0,penalty='l1')
clf1.fit(X_train,y_train.astype('int'))
y_pred1=clf1.predict(X_test)
logistic_accuracy=accuracy_score(y_test,y_pred1)   


from sklearn.tree import DecisionTreeClassifier
clf2= DecisionTreeClassifier(criterion='entropy')
clf2 = clf2.fit(X_train,y_train.astype('int'))
y_pred2= clf1.predict(X_test)

decision_accuracy=accuracy_score(y_test,y_pred2)   


C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [12]:
"""
7. 請以 Accuracy 評估同一模型在不同超參數設定下於訓練資料及測試資料上的表現
比較SVM模型在不同kernel(rbf、linear)下的accuracy
"""
clf3 = svm.SVC(kernel='rbf',C = 10) 
clf3.fit(X_train, y_train.astype('int')) 
y_pred3 = clf3.predict(X_test)

svm_accuracy1=accuracy_score(y_test,y_pred3)   


svmtwoallaccuracy={
    'kernel="linear"':[svm_accuracy],
    'kernel="rbf"':[svm_accuracy1],

}
df = pd.DataFrame(svmtwoallaccuracy)
df.index = ["accuracy"]
df

C:\Users\user\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


,"kernel=""linear""","kernel=""rbf"""
accuracy,0.805195,0.684416


In [14]:
"""
7. 請以 Accuracy 評估同一模型在不同超參數設定下於訓練資料及測試資料上的表現
比較LogisticRegression模型在不同penalty(l1、l2)下的accuracy
"""
clf4=LogisticRegression(penalty='l2')
clf4.fit(X_train,y_train.astype('int'))
y_pred4=clf4.predict(X_test)
logistic_accuracy1=accuracy_score(y_test,y_pred4)   


logistictwoallaccuracy={
    'penalty="l1"':[logistic_accuracy],
    'penalty="l2"':[logistic_accuracy1],

}
df = pd.DataFrame(logistictwoallaccuracy)
df.index = ["accuracy" ]
df

C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


,"penalty=""l1""","penalty=""l2"""
accuracy,0.833766,0.803896


In [15]:
"""
7. 請以 Accuracy 評估同一模型在不同超參數設定下於訓練資料及測試資料上的表現
比較decisiontree模型在不同criterion(gini、int)下的accuracy
"""
clf5= DecisionTreeClassifier(criterion='gini')
clf5 = clf5.fit(X_train,y_train.astype('int'))
y_pred5= clf5.predict(X_test)

decision_accuracy1=accuracy_score(y_test,y_pred5)   


decisiontwoallaccuracy={
    'criterion="entropy"':[decision_accuracy],
    'criterion="gini"':[decision_accuracy1]

}
df = pd.DataFrame(decisiontwoallaccuracy)
df.index = ["accuracy" ]
df

,"criterion=""entropy""","criterion=""gini"""
accuracy,0.833766,0.884416


In [16]:
#8. 請以 Accuracy 比較三個模型的表現 (6%)

from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

svm_accuracy = accuracy_score(y_test,y_pred)
logisticregression_accuracy = accuracy_score(y_test,y_pred1)
decisiontree_accuracy = accuracy_score(y_test, y_pred2)
allaccuracy={
    'accuracy':[svm_accuracy,logisticregression_accuracy,decisiontree_accuracy]
   
}
df = pd.DataFrame(allaccuracy)
df.index = ["svm_accuracy","logisticregression_accuracy","decisiontree_accuracy"]
df

,accuracy
svm_accuracy,0.805195
logisticregression_accuracy,0.833766
decisiontree_accuracy,0.833766


In [17]:
#9. 請列出三個模型的 Confusion matrix，並簡述其意義 
"""
左上角與右下角皆為預測對的數目
右上角與坐下角為預測錯誤的數目
準確率ACC:猜對的數目(TP+TN)/總數(TP+TN+FP+FN)
"""
print("svm confusion matrix")
print(confusion_matrix(y_test,y_pred1))


print("Logistic Regression confusion matrix")
print(confusion_matrix(y_test,y_pred2))


print("Decision Tree confusion matrix")
print(confusion_matrix(y_test,y_pred))


svm confusion matrix
[[482  45]
 [ 83 160]]
Logistic Regression confusion matrix
[[482  45]
 [ 83 160]]
Decision Tree confusion matrix
[[491  36]
 [114 129]]
